# Kaggle: Spaceship Titanic


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

## 数据预处理

In [2]:
data = pd.read_csv('data/train.csv')
print(data.head())

  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True  


In [3]:
# 删除Name
data.drop('Name', axis=1, inplace=True)
print(type(data['CryoSleep'][1]), type(data['VIP'][1]) )

<class 'bool'> <class 'bool'>


In [4]:
process_data = data.copy()

In [5]:
# 对于Roomservice和Foodservice，缺失值填充为0
age_mean = data['Age'].mean()
age_std = data['Age'].std()
age_null_count = data['Age'].isnull().sum()
age_random_list = np.random.normal(loc=age_mean, scale=age_std, size=age_null_count)
process_data['Age'][data['Age'].isnull()] = age_random_list
process_data['RoomService'] = data['RoomService'].fillna(0)
process_data['FoodCourt'] = data['FoodCourt'].fillna(0)
process_data['ShoppingMall'] = data['ShoppingMall'].fillna(0)
process_data['Spa'] = data['Spa'].fillna(0)
process_data['VRDeck'] = data['VRDeck'].fillna(0)

process_data.head()

/var/folders/74/cgypt0nj3714zwrsvp2_nyd80000gn/T/ipykernel_8473/3541105295.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  process_data['Age'][data['Age'].isnull()] = age_random_list
/var/folders/74/cgypt0nj3714zwrsvp2_nyd80000gn/T/ipyke

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [6]:
# 对于CryoSleep和VIP，缺失值填充为0
process_data['CryoSleep'] = data['CryoSleep'].fillna(0)
process_data['VIP'] = data['VIP'].fillna(0)
# 将VIP转换为0-1
process_data['VIP_New'] = process_data['VIP'].apply(lambda x: 1 if x else 0)
process_data['CryoSleep_New'] = process_data['CryoSleep'].apply(lambda x: 1 if x else 0)
process_data.drop(['CryoSleep', 'VIP'], axis=1, inplace=True)
# 修改名字
process_data.rename(columns={'VIP_New': 'VIP', 'CryoSleep_New': 'CryoSleep'}, inplace=True)
process_data.head(10)

process_data['Transported'] = data['Transported'].apply(lambda x: 1 if x else 0)

In [7]:
process_data.to_csv('data/train_process.csv', index=False)
process_data.head(10)

,PassengerId,HomePlanet,Cabin,Destination,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,VIP,CryoSleep
0,0001_01,Europa,B/0/P,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,0002_01,Earth,F/0/S,TRAPPIST-1e,24.0,109.0,9.0,25.0,549.0,44.0,1,0,0
2,0003_01,Europa,A/0/S,TRAPPIST-1e,58.0,43.0,3576.0,0.0,6715.0,49.0,0,1,0
3,0003_02,Europa,A/0/S,TRAPPIST-1e,33.0,0.0,1283.0,371.0,3329.0,193.0,0,0,0
4,0004_01,Earth,F/1/S,TRAPPIST-1e,16.0,303.0,70.0,151.0,565.0,2.0,1,0,0
5,0005_01,Earth,F/0/P,PSO J318.5-22,44.0,0.0,483.0,0.0,291.0,0.0,1,0,0
6,0006_01,Earth,F/2/S,TRAPPIST-1e,26.0,42.0,1539.0,3.0,0.0,0.0,1,0,0
7,0006_02,Earth,G/0/S,TRAPPIST-1e,28.0,0.0,0.0,0.0,0.0,0.0,1,0,1
8,0007_01,Earth,F/3/S,TRAPPIST-1e,35.0,0.0,785.0,17.0,216.0,0.0,1,0,0
9,0008_01,Europa,B/1/P,55 Cancri e,14.0,0.0,0.0,0.0,0.0,0.0,1,0,1


### 数据扩张

In [8]:
data = pd.read_csv('data/train_process.csv')
data.head(10)

,PassengerId,HomePlanet,Cabin,Destination,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,VIP,CryoSleep
0,0001_01,Europa,B/0/P,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,0002_01,Earth,F/0/S,TRAPPIST-1e,24.0,109.0,9.0,25.0,549.0,44.0,1,0,0
2,0003_01,Europa,A/0/S,TRAPPIST-1e,58.0,43.0,3576.0,0.0,6715.0,49.0,0,1,0
3,0003_02,Europa,A/0/S,TRAPPIST-1e,33.0,0.0,1283.0,371.0,3329.0,193.0,0,0,0
4,0004_01,Earth,F/1/S,TRAPPIST-1e,16.0,303.0,70.0,151.0,565.0,2.0,1,0,0
5,0005_01,Earth,F/0/P,PSO J318.5-22,44.0,0.0,483.0,0.0,291.0,0.0,1,0,0
6,0006_01,Earth,F/2/S,TRAPPIST-1e,26.0,42.0,1539.0,3.0,0.0,0.0,1,0,0
7,0006_02,Earth,G/0/S,TRAPPIST-1e,28.0,0.0,0.0,0.0,0.0,0.0,1,0,1
8,0007_01,Earth,F/3/S,TRAPPIST-1e,35.0,0.0,785.0,17.0,216.0,0.0,1,0,0
9,0008_01,Europa,B/1/P,55 Cancri e,14.0,0.0,0.0,0.0,0.0,0.0,1,0,1


In [9]:
def map_deck(deck):
    # Maps deck letters to numbers (A=1, B=2, C=3, etc.)
    if deck:  # Ensure deck is not None or empty
        return ord(deck) - ord('A') + 1
    return None

def map_room(room):
    # Convert room number directly to integer
    if room:  # Ensure room is not None or empty
        return int(room)
    return None

def map_info(info):
    # Map additional info characters to numbers (P=1, S=2, etc.)
    mapping = {'P': 1, 'S': 2}
    if info in mapping:  # Check if info is a recognized character
        return mapping[info]
    return None

# Update the function to handle missing values (NaN) safely in the Cabin column
def extract_cabin_parts(cabin):
    if pd.isna(cabin):
        return (-1, -1, -1)  # Return None for all parts if the cabin data is missing
    parts = cabin.split('/')
    deck = map_deck(parts[0]) if len(parts) > 0 and parts[0] else None
    room = map_room(parts[1]) if len(parts) > 1 and parts[1] else None
    info = map_info(parts[2]) if len(parts) > 2 and parts[2] else None
    return (deck, room, info)

# Apply the updated function and create new columns
data['Deck'], data['Room'], data['Info'] = zip(*data['Cabin'].apply(extract_cabin_parts))

data.drop('Cabin', axis=1, inplace=True)
# Show the new columns along with the original Cabin column
data.head(10)

,PassengerId,HomePlanet,Destination,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,VIP,CryoSleep,Deck,Room,Info
0,0001_01,Europa,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0,0,0,2,0,1
1,0002_01,Earth,TRAPPIST-1e,24.0,109.0,9.0,25.0,549.0,44.0,1,0,0,6,0,2
2,0003_01,Europa,TRAPPIST-1e,58.0,43.0,3576.0,0.0,6715.0,49.0,0,1,0,1,0,2
3,0003_02,Europa,TRAPPIST-1e,33.0,0.0,1283.0,371.0,3329.0,193.0,0,0,0,1,0,2
4,0004_01,Earth,TRAPPIST-1e,16.0,303.0,70.0,151.0,565.0,2.0,1,0,0,6,1,2
5,0005_01,Earth,PSO J318.5-22,44.0,0.0,483.0,0.0,291.0,0.0,1,0,0,6,0,1
6,0006_01,Earth,TRAPPIST-1e,26.0,42.0,1539.0,3.0,0.0,0.0,1,0,0,6,2,2
7,0006_02,Earth,TRAPPIST-1e,28.0,0.0,0.0,0.0,0.0,0.0,1,0,1,7,0,2
8,0007_01,Earth,TRAPPIST-1e,35.0,0.0,785.0,17.0,216.0,0.0,1,0,0,6,3,2
9,0008_01,Europa,55 Cancri e,14.0,0.0,0.0,0.0,0.0,0.0,1,0,1,2,1,1


In [10]:
data['Destination'] = data['Destination'].apply(lambda x: 1 if x == 'TRAPPIST-1e' else (2 if x == 'PSO J318.5-22' else (3 if x == '55 Cancri e' else 0)))
data['HomePlanet'] = data['HomePlanet'].apply(lambda x: 1 if x == 'Earth' else (2 if x == 'Europa' else (3 if x == 'Mars' else 0)))

print(data['HomePlanet'].value_counts())
print(data['Destination'].value_counts())

HomePlanet
1    4602
2    2131
3    1759
0     201
Name: count, dtype: int64
Destination
1    5915
3    1800
2     796
0     182
Name: count, dtype: int64


In [11]:
# 检查缺失值


In [13]:
if data.isnull().sum().sum() == 0:
    data.to_csv('data/trainData.csv', index=False)
    print('Data saved to file.')
else:
    print('Data still contains missing values.')

Data saved to file.


# 对测试集进行数据处理

In [14]:
test_data = pd.read_csv('data/test.csv')
test_data.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
5,0027_01,Earth,False,F/7/P,TRAPPIST-1e,31.0,False,0.0,1615.0,263.0,113.0,60.0,Karlen Ricks
6,0029_01,Europa,True,B/2/P,55 Cancri e,21.0,False,0.0,NaN,0.0,0.0,0.0,Aldah Ainserfle
7,0032_01,Europa,True,D/0/S,TRAPPIST-1e,20.0,False,0.0,0.0,0.0,0.0,0.0,Acrabi Pringry
8,0032_02,Europa,True,D/0/S,55 Cancri e,23.0,False,0.0,0.0,0.0,0.0,0.0,Dhena Pringry
9,0033_01,Earth,False,F/7/S,55 Cancri e,24.0,False,0.0,639.0,0.0,0.0,0.0,Eliana Delazarson


In [15]:
process_test = test_data.copy()
process_test['HomePlanet'] = test_data['HomePlanet'].apply(lambda x: 1 if x == 'Earth' else (2 if x == 'Europa' else (3 if x == 'Mars' else 0)))
process_test['CryoSleep'] = test_data['CryoSleep'].fillna(0)
process_test['CryoSleep'] = test_data['CryoSleep'].apply(lambda x: 1 if x else 0)
process_test['VIP'] = test_data['VIP'].fillna(0)
process_test['VIP'] = test_data['VIP'].apply(lambda x: 1 if x else 0)
process_test['RoomService'] = test_data['RoomService'].fillna(0)
process_test['FoodCourt'] = test_data['FoodCourt'].fillna(0)
process_test['ShoppingMall'] = test_data['ShoppingMall'].fillna(0)
process_test['VRDeck'] = test_data['VRDeck'].fillna(0)
process_test['Spa'] = test_data['Spa'].fillna(0)
process_test['Destination'] = test_data['Destination'].apply(lambda x: 1 if x == 'TRAPPIST-1e' else (2 if x == 'PSO J318.5-22' else (3 if x == '55 Cancri e' else 0)))
process_test['Age'] = test_data['Age'].fillna(age_mean)

In [16]:
process_test.drop('Name', axis=1, inplace=True)
# Apply Cabin extraction function to the test data
process_test['Deck'], process_test['Room'], process_test['Info'] = zip(*process_test['Cabin'].apply(extract_cabin_parts))
process_test.drop('Cabin', axis=1, inplace=True)
process_test.head(10)

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Room,Info
0,0013_01,1,1,1,27.0,0,0.0,0.0,0.0,0.0,0.0,7,3,2
1,0018_01,1,0,1,19.0,0,0.0,9.0,0.0,2823.0,0.0,6,4,2
2,0019_01,2,1,3,31.0,0,0.0,0.0,0.0,0.0,0.0,3,0,2
3,0021_01,2,0,1,38.0,0,0.0,6652.0,0.0,181.0,585.0,3,1,2
4,0023_01,1,0,1,20.0,0,10.0,0.0,635.0,0.0,0.0,6,5,2
5,0027_01,1,0,1,31.0,0,0.0,1615.0,263.0,113.0,60.0,6,7,1
6,0029_01,2,1,3,21.0,0,0.0,0.0,0.0,0.0,0.0,2,2,1
7,0032_01,2,1,1,20.0,0,0.0,0.0,0.0,0.0,0.0,4,0,2
8,0032_02,2,1,3,23.0,0,0.0,0.0,0.0,0.0,0.0,4,0,2
9,0033_01,1,0,3,24.0,0,0.0,639.0,0.0,0.0,0.0,6,7,2


In [17]:
if process_test.isnull().sum().sum() == 0:
    process_test.to_csv('data/testData.csv', index=False)
    print('Test data saved to file.')
else:
    print('Test data still contains missing values.')

Test data saved to file.
